In [ ]:
# PART 1

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
# set theurl and endpoint we want to hit
url = 'https://api.yelp.com/v3/businesses/search'
url2 = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [3]:
client_id = 'z5FPRiFN1Th8uQRPbWSEIg'
api_key= 'GxKB3hPDVkON6lH-yP1uhD9y0k6Rl9UYkGRugTXR4Es3bCatBMZtU0MMKiVzVJkSz3oThmpXXhRMFCVDfuXMfnnSAb1TytjQvlOJEJaWxc2RX4Vc_CweSLQKnHb2X3Yx'

In [4]:
# from  keys  import  client_id, api_key

In [5]:
print(api_key)

GxKB3hPDVkON6lH-yP1uhD9y0k6Rl9UYkGRugTXR4Es3bCatBMZtU0MMKiVzVJkSz3oThmpXXhRMFCVDfuXMfnnSAb1TytjQvlOJEJaWxc2RX4Vc_CweSLQKnHb2X3Yx


In [6]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [23]:
term = 'restaurant'
location = 'San Francisco'
categories = 'cafe'
radius = 3500

In [24]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "radius": radius,
                "limit": 50,
#                 "offset":offset
            }

In [25]:
url_params

{'term': 'restaurant',
 'location': 'San+Francisco',
 'categories': 'cafe',
 'radius': 3500,
 'limit': 50}

In [26]:
response = requests.get(url, headers=headers, params=url_params)

In [27]:
response.status_code

200

In [28]:
data = response.json()
data['businesses'][10]

{'id': 'PgJV1wOtwTuoUz_n5Mcyxw',
 'alias': 'tacos-el-patrón-san-francisco-2',
 'name': 'Tacos El Patrón',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/fkgyZbyYP7evChd0trFinA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/tacos-el-patr%C3%B3n-san-francisco-2?adjust_creative=z5FPRiFN1Th8uQRPbWSEIg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=z5FPRiFN1Th8uQRPbWSEIg',
 'review_count': 415,
 'categories': [{'alias': 'tacos', 'title': 'Tacos'}],
 'rating': 4.0,
 'coordinates': {'latitude': 37.748949, 'longitude': -122.416138},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '1500 S Van Ness Ave',
  'address2': 'Ste 100',
  'address3': '',
  'city': 'San Francisco',
  'zip_code': '94110',
  'country': 'US',
  'state': 'CA',
  'display_address': ['1500 S Van Ness Ave',
   'Ste 100',
   'San Francisco, CA 94110']},
 'phone': '+14158297315',
 'display_phone': '(415) 829-7315',
 'distance': 2224.9476542375687}

In [29]:
# PART 2

In [30]:
import sqlite3
import pandas as pd

In [31]:
conn = sqlite3.connect('yelp.db')
cursor = conn.cursor()

In [32]:
cursor.execute("""Create TABLE businesses(
                                businessid TEXT,
                                rating NUMERIC,
                                price TEXT,
                                name TEXT,
                                review_count INTEGER,
                                address TEXT,
                                city TEXT,
                                metro_area TEXT,
                                zip_code TEXT
                                )
                """)
conn.commit()

OperationalError: table businesses already exists

In [17]:
cursor.execute("""Create TABLE reviews(
                                businessid TEXT,
                                reviewID INTEGER,
                                rating NUMERIC,
                                time_created TEXT,
                                text TEXT
                                )
                """)
conn.commit()

In [33]:
def yelp_call(url, url_params, api_key):
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    return data

In [34]:
results = yelp_call(url, url_params, api_key)

In [35]:
def parse_results(results):
    
    parsed_results = []
    count = 0
    
    for business in results['businesses']:
        for item in ['businessid', 'rating', 'price', 'name', 'review_count', 'address', 'city', 'metro_area', 'zip_code']:
            if item not in business:
                business[item] = np.nan
            else:
                business[item]
    
        biz_tuple = (business['id'],
                business['rating'],
                business['price'],
                business['name'],
                business['review_count'],
                business['location']['display_address'][0],
                business['location']['city'],
                location,
                business['location']['zip_code']
                    )
        count += 1
        parsed_results.append(biz_tuple)
    
    return parsed_results

In [36]:
def db_insert(conn, cursor, parse_results):
# for business in parse_results:
    add_business = ("""
    INSERT INTO businesses (businessid, rating, price, name, review_count, address, city, metro_area, zip_code) 
    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)
    """)
    cursor.executemany(add_business, parse_results)
    conn.commit()

In [37]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url, url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(conn, cursor, parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50
    
print(cur)

1000


In [38]:
query = ("SELECT businessid FROM businesses")
cursor.execute(query)
business_ID_list = cursor.fetchall()
business_ID_list

[('6i7KnRylm4bhPIApDFSVCg',),
 ('h1R2iKYdm2lwukzMJvJqDw',),
 ('rr-hXB0ZqXLzUP97yftp0g',),
 ('AIhWsp8yNfKZfaX9d9Yajg',),
 ('qAAoilHU25Qr45FOQlA19g',),
 ('JYGPEUZy4k5ObXGIjcD3DA',),
 ('IzRINfSLBr6k_dq1e2WNew',),
 ('HuQtriV_ApipXfM8xax33w',),
 ('DrToq9357afdpOyO5w0Y5w',),
 ('MlmcOkwaNnxl3Zuk6HsPCQ',),
 ('qCrSlf_TSVxITOYIBXGdJQ',),
 ('TsVu9GJQwuFBZVP_sg2k7A',),
 ('q8buTNBt3FU9vie0D5OOxA',),
 ('jisjJNoV5jzfAMIYKn7y5A',),
 ('vs8USEc1A480H8MnmFKs5A',),
 ('DoyIC0ElqglrlHBKDxXOwg',),
 ('-3S1PdyDbSQcqkIaX8VKrQ',),
 ('BEItepse0gBSfkJ4sb_k-g',),
 ('IxuAHGu-eQ4fZJZTdzGTYQ',),
 ('YCEgBqClpWXQ3leoAhWjoA',),
 ('TWHGJkTAbF22hvXeReQp9w',),
 ('Fb3WueRF1ekhs72psinq5w',),
 ('0SerWqGwzNWpTA2RBLVdUg',),
 ('-ZcCFCdqMwOPXItB3REecA',),
 ('KtEMG1Aln3vQzte92QJxlw',),
 ('YoQn8MtOimQbqi839TmeZw',),
 ('y4X0KfHJ5XWwxMIRukVr1Q',),
 ('1ZEUvoDlg7ixlwwlK2a3mQ',),
 ('LLj451UZrvt3SkZVrEZ4ew',),
 ('HTFByVH44biQsc_R8sEOjg',),
 ('23Qrcz2i9e2e8MstUIIO0A',),
 ('h_VkRMpSrtjdKDxUJ7sDOQ',),
 ('H9ZlIqEp930ZRniH6VRp8w',),
 ('frRPjJ0

In [39]:
type(business_ID_list[0][0])

str

In [ ]:
# pd.read_sql("SELECT businessid FROM businesses", conn)

In [ ]:
# pd.DataFrame(business_ID_list)

In [54]:
biz_id_list = []
for biz in business_ID_list:
    biz_id = biz[0]
    biz_id_list.append(biz_id)

In [55]:
biz_id_list

['6i7KnRylm4bhPIApDFSVCg',
 'h1R2iKYdm2lwukzMJvJqDw',
 'rr-hXB0ZqXLzUP97yftp0g',
 'AIhWsp8yNfKZfaX9d9Yajg',
 'qAAoilHU25Qr45FOQlA19g',
 'JYGPEUZy4k5ObXGIjcD3DA',
 'IzRINfSLBr6k_dq1e2WNew',
 'HuQtriV_ApipXfM8xax33w',
 'DrToq9357afdpOyO5w0Y5w',
 'MlmcOkwaNnxl3Zuk6HsPCQ',
 'qCrSlf_TSVxITOYIBXGdJQ',
 'TsVu9GJQwuFBZVP_sg2k7A',
 'q8buTNBt3FU9vie0D5OOxA',
 'jisjJNoV5jzfAMIYKn7y5A',
 'vs8USEc1A480H8MnmFKs5A',
 'DoyIC0ElqglrlHBKDxXOwg',
 '-3S1PdyDbSQcqkIaX8VKrQ',
 'BEItepse0gBSfkJ4sb_k-g',
 'IxuAHGu-eQ4fZJZTdzGTYQ',
 'YCEgBqClpWXQ3leoAhWjoA',
 'TWHGJkTAbF22hvXeReQp9w',
 'Fb3WueRF1ekhs72psinq5w',
 '0SerWqGwzNWpTA2RBLVdUg',
 '-ZcCFCdqMwOPXItB3REecA',
 'KtEMG1Aln3vQzte92QJxlw',
 'YoQn8MtOimQbqi839TmeZw',
 'y4X0KfHJ5XWwxMIRukVr1Q',
 '1ZEUvoDlg7ixlwwlK2a3mQ',
 'LLj451UZrvt3SkZVrEZ4ew',
 'HTFByVH44biQsc_R8sEOjg',
 '23Qrcz2i9e2e8MstUIIO0A',
 'h_VkRMpSrtjdKDxUJ7sDOQ',
 'H9ZlIqEp930ZRniH6VRp8w',
 'frRPjJ03Ie5h9EPhFhjNIw',
 'pjh40JY5YwWeV8aKhkXERg',
 '9FuxqbYXM0vaPnzrQ7B_qQ',
 'ru7TAEo7lkbxMmkbFRGFMw',
 

In [59]:
def yelp_call_reviews(ID):
    url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(ID)
    response = requests.get(url, headers=headers)
    data = response.json()
    return data
#     if 'reviews' in data:
#         return data['reviews']
#     else:
#         return None

In [60]:
yelp_call_reviews('GbAZbcCR969hfa4kCQqRcw')

{'reviews': [{'id': 'DjaiVl_LKXsTHHrZqxNZ7g',
   'url': 'https://www.yelp.com/biz/eat-this-cafe-los-angeles?adjust_creative=z5FPRiFN1Th8uQRPbWSEIg&hrid=DjaiVl_LKXsTHHrZqxNZ7g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=z5FPRiFN1Th8uQRPbWSEIg',
   'text': 'Bottomless mimosas?! Sign me uppp!!\n\nI got a chance to try Eat This Cafe pre-COVID and it was a great experience! The line does get SUPER long though since...',
   'rating': 5,
   'time_created': '2020-12-13 21:49:12',
   'user': {'id': '22GVpNvRujO8KRBGpEcgqg',
    'profile_url': 'https://www.yelp.com/user_details?userid=22GVpNvRujO8KRBGpEcgqg',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/KrdyzO-IyTACWrJFoHtEJg/o.jpg',
    'name': 'Diana L.'}},
  {'id': '2bC581maEO0vq-_RadtU_g',
   'url': 'https://www.yelp.com/biz/eat-this-cafe-los-angeles?adjust_creative=z5FPRiFN1Th8uQRPbWSEIg&hrid=2bC581maEO0vq-_RadtU_g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=z5FPRiFN1Th8uQRPb

In [61]:
biz = biz_id_list[0]

In [62]:
# biz

In [63]:
api_results = yelp_call_reviews(biz)
api_results

{'reviews': [{'id': 'I23fs_C6pvbuFFhRR2bCrQ',
   'url': 'https://www.yelp.com/biz/daves-hot-chicken-los-angeles?adjust_creative=z5FPRiFN1Th8uQRPbWSEIg&hrid=I23fs_C6pvbuFFhRR2bCrQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=z5FPRiFN1Th8uQRPbWSEIg',
   'text': 'wowwowowww. first off, how did I not know about this place sooner? Well I knew about it, but I always held off trying it. LA is saturated with hot chicken...',
   'rating': 5,
   'time_created': '2020-12-10 11:47:57',
   'user': {'id': 'OpKz4liDaShMItXWMuvikw',
    'profile_url': 'https://www.yelp.com/user_details?userid=OpKz4liDaShMItXWMuvikw',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/CwNxwx_eJ78V2mHJAiT7Mg/o.jpg',
    'name': 'Rochella Anne R.'}},
  {'id': 'bkcmIvgq0z7G6G2tGysqeQ',
   'url': 'https://www.yelp.com/biz/daves-hot-chicken-los-angeles?adjust_creative=z5FPRiFN1Th8uQRPbWSEIg&hrid=bkcmIvgq0z7G6G2tGysqeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=z5FP

In [64]:
# api_results

In [65]:
tuple_list = []
for x in api_results:  
    review_tuple = (
                    x['id'],
                    x['text'], 
                    x['rating'], 
                    x['time_created'],
                   )
    tuple_list.append(review_tuple)

# tuple_list

TypeError: string indices must be integers

In [68]:
def parse_review(businessID, review_results):
    tuple_lists = [] 
    
    if 'error' in review_results:
        if review_results['error']['code'] == 'BUSINESS_UNAVAILABLE':
            return None
    else:
        for review in review_results['reviews']:
            for item in ['text','rating','time_created']:
                if item not in review:
                    review[item] = np.nan
            review_tuple = (businessID,
                            review['id'], 
                            review['rating'],
                            review['time_created'],
                            review['text'],
                          )
            tuple_lists.append(review_tuple)
        
    return tuple_lists

In [69]:
parse_review(biz, api_results)

[('6i7KnRylm4bhPIApDFSVCg',
  'I23fs_C6pvbuFFhRR2bCrQ',
  5,
  '2020-12-10 11:47:57',
  'wowwowowww. first off, how did I not know about this place sooner? Well I knew about it, but I always held off trying it. LA is saturated with hot chicken...'),
 ('6i7KnRylm4bhPIApDFSVCg',
  'bkcmIvgq0z7G6G2tGysqeQ',
  5,
  '2020-10-27 10:20:20',
  "UHM YUM.  If you've never hopped on this chicken eating trend let me tell you, you need to! Speaking from first hand experience. \n\nFood - amazing. I got a..."),
 ('6i7KnRylm4bhPIApDFSVCg',
  'UHFL1uf_ZLR8iWflNJ9Uug',
  5,
  '2020-08-05 14:30:01',
  "FINALLY!!!  No line!!!  I'm FINALLY going to see what all the talk is about.  \n\nLocated in a small, not very attractive, mini strip mall with shared...")]

In [70]:
def review_insert(conn, cursor, parse_reviews):
    add_reviews = ("""
    INSERT INTO reviews(businessid, reviewID, rating, time_created, text) 
    VALUES(?, ?, ?, ?, ?)
    """)
    cursor.executemany(add_reviews, parse_reviews)
    conn.commit()

In [71]:
biz_id_list.index('ACA2q2bYEy8eDzvoad9SZQ')

1224

In [72]:
biz_id_list[1746]

'FroRRBJvJHosq8wM7BnmCQ'

In [ ]:
"""SELECT reviewID, count(*) as C
FROM reviews 
group by reviewID 
HAVING C > 1"""

In [ ]:
# query unique biz ids in reveiw table

# write a list comprehension to give you biz 

In [74]:
for biz in biz_id_list:
    review_results = yelp_call_reviews(biz)
    if review_results == None:
        pass
    parsed_reviews = parse_review(biz, review_results)
    review_insert(conn, cursor, parsed_reviews)
    

In [ ]:
yelp_call_reviews('Boa2T4-JplIX7ATmrnzI5w')

In [ ]:
# SELECT reviewID, count(*) as C
# FROM reviews 
# group by reviewID 
# HAVING C > 1;

In [ ]:
# cursor.execute("""ALTER TABLE reviews
# RENAME COLUMN reviewID TO texto;""")


In [ ]:
# cursor.execute("""ALTER TABLE reviews
# RENAME COLUMN  text TO reviewID;""")

In [ ]:
# cursor.execute("""ALTER TABLE reviews
# RENAME COLUMN texto TO text;""")


In [75]:
# Which are the 5 most reviewed businesses?
cursor.execute("""
SELECT name, review_count
FROM businesses
ORDER BY review_count DESC
LIMIT 5
""").fetchall()

[("Brenda's French Soul Food", 11055),
 ('Tartine Bakery & Cafe', 8014),
 ("Pink's Hot Dogs", 7163),
 ('Burma Superstar', 6837),
 ('Republique', 5628)]

In [76]:
# What is the highest rating recieved in your data set and how many businesses have that rating?
cursor.execute("""
SELECT name, rating
FROM businesses
WHERE rating = 5""").fetchall()

[('Woo Hyang Woo', 5),
 ('FishPop', 5),
 ('Dashijiru', 5),
 ('Casablanca Moroccan kitchens', 5),
 ('Soseji', 5),
 ('Blue Bird Burgers', 5),
 ('Tang Huo Kung Fu Spicy Hot Pot', 5),
 ("Gigi's", 5),
 ('Chef Kang Korean Taco', 5),
 ('Love Hour', 5),
 ('Salmon Talk', 5),
 ('Sulga House of Beef Bone Soup', 5),
 ('Viet On Melrose', 5),
 ('Chef Arango Indian Restaurant', 5),
 ('CROQUÉ', 5),
 ('Sabor Nicaragüense Restaurant', 5),
 ('Noya Kitchen', 5),
 ('Colibri Cafe', 5),
 ('Saikai Ramen Bar', 5),
 ('BBQ + RICE', 5),
 ('SYTK Sullungtang', 5),
 ('LAces', 5),
 ('Burgerdaddy', 5),
 ('La Piramide Salvadorena Restaurant', 5),
 ('Lemme Smash Burgers', 5),
 ('Sip & Chick', 5),
 ('Gokobol', 5),
 ('Bondak', 5),
 ('Three Borders Brunch & Grill', 5),
 ('Everyday Pocha', 5),
 ('Lucy Ethiopian Restaurant', 5),
 ('Tacos Guelaguetza', 5),
 ('Fishbone Seafood Hollywood', 5),
 ("Lieder's on La Brea", 5),
 ('Frydays Hot Chicken', 5),
 ('Don Don Lee Soondae', 5),
 ('Rice Guys', 5),
 ("Nico's Pizza", 5),
 ('Boran

In [77]:
cursor.execute("""
SELECT name, rating
FROM businesses 
WHERE rating IN 
(SELECT MAX(rating) FROM businesses)""").fetchall()

[('Woo Hyang Woo', 5),
 ('FishPop', 5),
 ('Dashijiru', 5),
 ('Casablanca Moroccan kitchens', 5),
 ('Soseji', 5),
 ('Blue Bird Burgers', 5),
 ('Tang Huo Kung Fu Spicy Hot Pot', 5),
 ("Gigi's", 5),
 ('Chef Kang Korean Taco', 5),
 ('Love Hour', 5),
 ('Salmon Talk', 5),
 ('Sulga House of Beef Bone Soup', 5),
 ('Viet On Melrose', 5),
 ('Chef Arango Indian Restaurant', 5),
 ('CROQUÉ', 5),
 ('Sabor Nicaragüense Restaurant', 5),
 ('Noya Kitchen', 5),
 ('Colibri Cafe', 5),
 ('Saikai Ramen Bar', 5),
 ('BBQ + RICE', 5),
 ('SYTK Sullungtang', 5),
 ('LAces', 5),
 ('Burgerdaddy', 5),
 ('La Piramide Salvadorena Restaurant', 5),
 ('Lemme Smash Burgers', 5),
 ('Sip & Chick', 5),
 ('Gokobol', 5),
 ('Bondak', 5),
 ('Three Borders Brunch & Grill', 5),
 ('Everyday Pocha', 5),
 ('Lucy Ethiopian Restaurant', 5),
 ('Tacos Guelaguetza', 5),
 ('Fishbone Seafood Hollywood', 5),
 ("Lieder's on La Brea", 5),
 ('Frydays Hot Chicken', 5),
 ('Don Don Lee Soondae', 5),
 ('Rice Guys', 5),
 ("Nico's Pizza", 5),
 ('Boran

In [78]:
cursor.execute("""
SELECT rating, count(rating)
FROM businesses
GROUP BY rating
ORDER BY rating DESC
LIMIT 1
""").fetchall()

[(5, 171)]

In [79]:
# What percentage of businesses have a rating greater than or 4.5?
n=cursor.execute("""SELECT round(count(rating), 2) * 100 / 
(SELECT count(businessid) FROM businesses)
FROM BUSINESSES
WHERE rating > 4.5""").fetchall()

n[0][0]

8.55

In [80]:
# What percentage of businesses have a rating less than 3?
n=cursor.execute("""SELECT round(count(rating), 2) * 100 / 
(SELECT count(businessid) FROM businesses)
FROM BUSINESSES
WHERE rating < 3""").fetchall()

n[0][0]

1.0

In [81]:
# What is the average rating of businesses that have a price label of one dollar sign? Two dollar signs? Three dollar signs?
x=cursor.execute("""SELECT
                   price, AVG(rating)
                   FROM businesses
                   WHERE price = '$' OR price = '$$' OR price = '$$$'
                   GROUP BY price
                   """).fetchall()


In [ ]:
# Return the text of the reviews for the most reviewed business.

cursor.execute("""SELECT text
                FROM reviews
                WHERE businessid = (SELECT businessid 
                FROM businesses 
                ORDER BY review_count DESC
                LIMIT 1)""").fetchall()

In [ ]:
# Return the name of the business with the most recent review.

cursor.execute("""SELECT name
                FROM businesses
                WHERE businessid = (SELECT businessid 
                FROM reviews 
                ORDER BY time_created DESC
                LIMIT 1)""").fetchall()

In [ ]:
# Find the highest rated business and return text of the most recent review. If multiple business have the same rating, 
# select the business with the most reviews.
cursor.execute("""SELECT b.name, r.text, b.rating, r.time_created, b.review_count
                FROM reviews AS r
                LEFT JOIN businesses AS b 
                ON b.businessid = r.businessid
                ORDER BY b.rating DESC, b.review_count DESC, r.time_created DESC
                LIMIT 1""").fetchall()

In [ ]:
# Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, 
# select the business with the least review

cursor.execute("""SELECT b.name, r.text, b.rating, r.time_created, b.review_count
                FROM reviews AS r
                LEFT JOIN businesses AS b 
                ON b.businessid = r.businessid
                ORDER BY b.rating, b.review_count, r.time_created DESC
                LIMIT 1""").fetchall()